In [34]:
import spacy
import pandas as pd
import numpy as np
#from fuzzywuzzy import fuzz
from rapidfuzz import fuzz
from tqdm import notebook
from datetime import datetime
import ast
import timeit
import tensorflow as tf
from tensorflow.compat.v1 import Session
from tensorflow.python.saved_model import loader
import os
import csv
import re
import logging
import optparse
import dedupe
from unidecode import unidecode
notebook.tqdm.pandas()

In [35]:
nlp2 = spacy.load('../Spacy models/kfold_3', disable=['parser', 'tagger', 'textcat'])
model = 'rl models/better_current/70no'
if os.path.exists(model):
        print('reading from', model)
        with open(model, 'rb') as sf:
            linker = dedupe.StaticRecordLink(sf)

else:
    raise ValueError('Could not find model')
notary = pd.read_csv('../clean_data.csv')
notary['jaar'] = [x[0:4] for x in notary.datering]
voc = pd.read_csv('vocop_clustered_dutchrank.csv')


#voc = pd.read_csv('vocop-clustered-new.csv', sep='	')
#rangen = pd.read_excel('../vocop_rangen.xlsx', index_col=0)
#voc['dutch_rank'] = [translate_rank(x, rangen) for x in notebook.tqdm(voc['rank'].tolist())]

INFO:dedupe.api:((SimplePredicate: (commonFourGram, ship_out), SimplePredicate: (commonSixGram, name)), (SimplePredicate: (sameFiveCharStartPredicate, ship_return), TfidfNGramSearchPredicate: (0.8, ship_return)), (ExistsPredicate: (Exists, ship_return), SimplePredicate: (oneGramFingerprint, location)), (SimplePredicate: (commonFourGram, rank), SimplePredicate: (sameThreeCharStartPredicate, ship_return)), (SimplePredicate: (oneGramFingerprint, location), SimplePredicate: (sameSevenCharStartPredicate, rank)), (SimplePredicate: (commonSixGram, ship_return), SimplePredicate: (suffixArray, name)))


reading from rl models/better_current/70no


In [4]:
jaren = set([x for x in notary[notary.groupby("jaar")["jaar"].transform('size') > 2].jaar])
voc = voc[(voc.yearBeginService.isin(jaren)) | (voc.year_end_service_improved.isin(jaren))]

In [17]:
set(voc.shipOutward.str.lower().to_list() + voc.shipReturn.str.lower().to_list())


691

## NER functions

In [36]:
def NER(text, nlp):
    ''' Takes in a string and uses the specified NLP model to tag entities within the string.
        Returns all entities tagged as PERSON within the string.
    '''
    # Tag text
    doc = nlp(text)
    holder = []
    
    # Append all entities tagged as PERSON
    for ent in doc.ents:
        if ent.label_ == 'PERSON' and ' ' in ent.text and ent.text not in holder:
            switch = True
            for x in holder:
                if fuzz.ratio(x, ent.text) >= 90:
                    switch = False
            if switch:
                holder.append(ent.text)
#             holder.append(ent.text)
    return holder

## RLfunctions

In [37]:
def clustering(model_dir, notary, voc):
    linker = model_dir
    
    data_1 = notary
    data_2 = voc

    def descriptions():
        for dataset in (data_1, data_2):
            for record in dataset.values():
                yield record['description']
    
    # ## Blocking

    # ## Clustering

    # Find the threshold that will maximize a weighted average of our
    # precision and recall.  When we set the recall weight to 2, we are
    # saying we care twice as much about recall as we do precision.
    #
    # If we had more data, we would not pass in all the blocked data into
    # this function but a representative sample.

    try:
        linked_records = linker.join(data_1, data_2, 0.7)
    except:
        return None

    # ## Writing Results

    # Write our original data back out to a CSV with a new column called
    # 'Cluster ID' which indicates which records refer to each other.

    cluster_membership = {}
    for cluster_id, (cluster, score) in enumerate(linked_records):
        for record_id in cluster:
            cluster_membership[record_id] = {'Cluster ID': cluster_id,
                                             'Link Score': score}
    clusters = {}
    for x in cluster_membership:
        if cluster_membership[x]['Cluster ID'] not in clusters:
            clusters[cluster_membership[x]['Cluster ID']] = x
        else:
            clusters[cluster_membership[x]['Cluster ID']] = [clusters[cluster_membership[x]['Cluster ID']], x]
    
    matches = clusters
    return matches


In [80]:

def match_finder(row, match, distance, nlp):
    if len(match) >  1 and match[1] == 'schip':
        true = [match[0]]
        ship_check = True
    else:
        true = match
        ship_check = False
    doc = nlp(row.text)
    locs = []
    prev = 0
    for token in doc:
        for x in true:
            if type(x) == str:
                new_token = []
                split = x.split(' ')
                for y in range(len(split)):
                    if token.i <= doc[-len(split)].i:
                        new_token.append(token.nbor(y).text)
                new_token_text = ' '.join(new_token)
                if fuzz.ratio(x.lower(), new_token_text.lower()) >= distance:
                    result = (token.idx, token.idx + len(new_token_text))
                    if result not in locs:
                        if fuzz.ratio(row.text[result[0]:result[1]].lower(), x.lower()) >= distance:
                            locs.append(result)
#     if ship_check == True and locs == []:
#         knowledgebase = voc[(voc.yearBeginService.isin([row.jaar, str(int(row.jaar) - 1), str(int(row.jaar) + 1)])) | 
#                             (voc.year_end_service_improved.isin([row.jaar, str(int(row.jaar) - 1), str(int(row.jaar) + 1)]))
#                            ]
#         ships = set(knowledgebase.shipOutward.str.lower().to_list() + 
#                     knowledgebase.shipReturn.str.lower().to_list())
#         if len(match) > 2:
#             if str(match[2][0]).lower() in ships:
#                 ships.remove(str(match[2][0]).lower())
#             if str(match[2][1]).lower() in ships:
#                 ships.remove(str(match[2][1]).lower())
#         for token in doc:
#             if fuzz.ratio('schip', token.text.lower()) >= distance:
#                 for x in ships:
#                     if type(x) == str:
#                         new_token = []
#                         split = x.split(' ')
#                         for y in range(len(split)):
#                             if token.nbor().i <= doc[-len(split)].i:
#                                 new_token.append(token.nbor(1 + y).text)
#                         new_token_text = ' '.join(new_token)
#                         if fuzz.ratio(x.lower(), new_token_text.lower()) >= distance:
#                             result = (token.nbor().idx, token.nbor().idx + len(new_token_text))
#                             if result not in locs:
#                                 if fuzz.ratio(row.text[result[0]:result[1]].lower(), x.lower()) >= distance:
#                                     locs.append(result)
    entities = [row.text[x[0]:x[1]] for x in locs]
    return entities

def find_entities(row, x, column, nlp):
#     if column == 'shipOutward' or column == 'shipReturn':
#         if type(x[column]) != float:
#             match = match_finder(row, [x[column], 'schip', (x['shipOutward'], x['shipReturn'])], 80, nlp)
#         else:
#             match = match_finder(row, ['schip'], 80, nlp)
    if type(x[column]) != float: 
        match = match_finder(row, [x[column]], 80, nlp) 
    else:
        match = ''
    return match

def find_matches(name, knowledgebase, target_column, distance=90):
    ''' Takes in a string containing the name of a person and returns all possible matches from the
        knowledgebase based on fuzzy string matching.
    '''
    matches = knowledgebase[(knowledgebase[target_column].str.lower().astype(str)
                             .apply(fuzz.ratio, args=[name.lower()]) >= distance)]
    return matches

def convert_matches(matches, knowledgebase, nlp, df):
    data_n = {}
    data_v = {}
    c = 0
    for entity in notebook.tqdm(matches):
        row = df[df.uuid == entity[-1]].iloc[0]
        holder = []
        notary_date = datetime.strptime(row.datering, '%Y-%m-%d')

        for x in range(entity[1].shape[0]):
            try:
                date1 = datetime.strptime(entity[1]['date_begin_service_complete'].iloc[x], '%Y-%m-%d')
            except:
                date1 = datetime(year=1, month=1, day =1 )
            try:
                date2 = datetime.strptime(entity[1]['date_end_service_complete'].iloc[x], '%Y-%m-%d')
            except:
                date2 = datetime(year=1, month=1, day =1 )
            if abs((notary_date - date1).days) < 90 or abs((notary_date - date2).days) < 90:
                found_ship_out = find_entities(row, entity[1].iloc[x], 'shipOutward', nlp)
                found_ship_return = find_entities(row, entity[1].iloc[x], 'shipReturn', nlp)
                found_loc = find_entities(row, entity[1].iloc[x], 'placeOfOrigin', nlp)
                found_rank = find_entities(row, entity[1].iloc[x], 'dutch_rank', nlp)
                holder.append({'rank':found_rank, 'location':found_loc, 
                               'found_ship_return': found_ship_return, 'found_ship_out': found_ship_out})

                cluster = int(entity[1].iloc[x].VOCOP_id)
                name = str(entity[1].iloc[x].fullNameOriginal).lower()
                rang = str(entity[1].iloc[x].dutch_rank.lower).lower()
                loc = str(entity[1].iloc[x].placeOfOrigin).lower()
                ship_out = str(entity[1].iloc[x].shipOutward).lower() 
                ship_return = str(entity[1].iloc[x].shipReturn).lower()
                data_v[cluster] = {'name':name, 'rank':rang, 'location':loc, 
                                   'ship_out': ship_out, 'ship_return': ship_return}

        name = entity[0].lower()
        if holder == []:
            rang = None
            location = None
            ship_out= None
            ship_return = None
        else:
            rang = ' | '.join(set([y.lower() for x in holder for y in x['rank']]))
            if rang == '':
                rang = None
            location = ' | '.join(set([y.lower() for x in holder for y in x['location']]))
            if location == '':
                location = None
            ship_out = ' | '.join(set([y.lower() for x in holder for y in x['found_ship_out']]))
            if ship_out == '':
                ship_out = None
            ship_return = ' | '.join(set([y.lower() for x in holder for y in x['found_ship_return']]))
            if ship_return == '':
                ship_return = None
        data_n[entity[-1] + '_' + str(c)] = {'name':name, 'rank':rang, 'location':location, 
                                             'ship_out': ship_out, 'ship_return': ship_return}
        c += 1
    d_n = {'index': [x for x in data_n], 'name': [data_n[x]['name'] for x in data_n], 
           'rank':[data_n[x]['rank'] for x in data_n], 'location':[data_n[x]['location'] for x in data_n], 
           'ship_out': [data_n[x]['ship_out'] for x in data_n], 
           'ship_return': [data_n[x]['ship_return'] for x in data_n]}

    d_v = {'index': [x for x in data_v], 'name': [data_v[x]['name'] for x in data_v], 
           'rank':[data_v[x]['rank'] for x in data_v], 'location':[data_v[x]['location'] for x in data_v], 
           'ship_out': [data_v[x]['ship_out'] for x in data_v], 
           'ship_return': [data_v[x]['ship_return'] for x in data_v]}

    dedupe_notary = pd.DataFrame(d_n).set_index('index')
    dedupe_notary = dedupe_notary.to_dict('index')
    
#     dedupe_notary.index.name = 'index'
#     dedupe_notary.to_csv('converted_notary.csv')
    
    dedupe_voc = pd.DataFrame(d_v).set_index('index')
    dedupe_voc = dedupe_voc.to_dict('index')
#     dedupe_voc.index.name = 'index'
#     dedupe_voc.to_csv('converted_voc.csv')
#     return 'converted_notary.csv', 'converted_voc.csv'
    return dedupe_notary, dedupe_voc

def RL(entity, knowledgebase, model_dir, nlp, df):
    ''' Takes in a string containing an entity and returns either a match within the specified pandas 
        knowledgebase or None if no match is present.
    ''' 
    # Find and narrow down matches
#     begin_date = []
#     end_date = []
#     for x in notebook.tqdm(range(knowledgebase.shape[0])):
#         try:
#             date1 = datetime.strptime(knowledgebase['date_begin_service_complete'].iloc[x], '%Y-%m-%d')
#         except:
#             date1 = datetime(year=1, month=1, day =1 )
#         begin_date.append(date1)
#         try:
#             date2 = datetime.strptime(knowledgebase['date_end_service_complete'].iloc[x], '%Y-%m-%d')
#         except:
#             date2 = datetime(year=1, month=1, day =1 )
#         end_date.append(date2)
    

#     knowledgebase['converted_begin_date'] = begin_date
#     knowledgebase['converted_end_date'] = end_date
#     possible_matches = []
#     for entry in notebook.tqdm(entity):
#         notary_date = datetime.strptime(entry[-1], '%Y-%m-%d')
#         rows = knowledgebase[(knowledgebase.converted_begin_date.apply(
#                               lambda x: abs(x - notary_date).days) < 90) | 
#                              (knowledgebase.converted_end_date.apply(
#                               lambda x: abs(x - notary_date).days) < 90)]
#         for person in entry[0]:
#             match = find_matches(person, rows, 'fullNameOriginal')
#             possible_matches.append((person, match, entry[1]))
            
        
    possible_matches = [(x, find_matches(x, knowledgebase[(knowledgebase.yearBeginService
                                                           .isin([y[2], str(int(y[2]) - 1), 
                                                                  str(int(y[2]) + 1)])) | 
                                                          (knowledgebase.year_end_service_improved
                                                           .isin([y[2], str(int(y[2]) - 1), 
                                                                  str(int(y[2]) + 1)]))], 
                                         'fullNameOriginal'), y[1]) 
                        for y in notebook.tqdm(entity) for x in y[0]]


    found_matches = 0
    all_ents = []
    for x in possible_matches:
        holder = []
        rows = final_df[final_df.vocop_match != '0'][(final_df.uuid == x[2]) & (final_df.vocop_match != '0') ]
        
        for match in rows.vocop_match:
            for y in x[1].VOCOP_id:

                if str(voc.loc[ast.literal_eval(match)[1]].VOCOP_id) == str(y):
                    holder.append(match)
                    all_ents.append(match)
        found_matches += len(holder)
    print("RL preselection has found " + str(found_matches) + " of the 101 matches.")
    
    all_clusters = []
    converted_entities, converted_matches = convert_matches(possible_matches, knowledgebase, nlp, df)
    clusters = clustering(model_dir, converted_entities, converted_matches)
    if clusters != None:
        for x in clusters:
            all_clusters.append(clusters[x])

    matched_matches = 0
    for x in all_clusters:
        holder = []
        row = final_df[final_df.vocop_match != '0'][(final_df.uuid == x[0].split('_', 1)[0]) & (final_df.vocop_match != '0') ]
        for match in row.vocop_match:
            if str(x[1]) == str(voc.loc[ast.literal_eval(match)[1]].VOCOP_id) and str(x[1]) not in holder:
                holder.append(str(x[1]))
        matched_matches += len(holder)
    print("RL model has matched " + str(matched_matches) + " of the 101 matches.")
    return clusters

## Final Function

In [83]:
def Pipeline(df, knowledgebase, NER_model, RL_model):
    holder = []
    entities = [(NER(x.text, NER_model), x.uuid, x.jaar) for x in notebook.tqdm(df.itertuples(), total=df.shape[0])]
    
    found_entities = 0
    for x in entities:
        holder = []
        rows = final_df[final_df.vocop_match != '0'][(final_df.uuid == x[1]) & (final_df.vocop_match != '0') ]
        for name in rows.name:
            for y in x[0]:
                if fuzz.ratio(name.lower(), y.lower()) >= 80 and name not in holder:
                    holder.append(name)
        found_entities += len(holder)
    print("NER model has found " + str(found_entities) + " of the 101 entities.")
    
    matches = RL(entities, knowledgebase, RL_model, NER_model, df)
    print('RL model has made a total of ' + str(len(matches)) + ' matches')
    return matches


In [84]:
start_time = timeit.default_timer()
x = Pipeline(notary[notary.uuid.isin(final_df.uuid)], voc, nlp2, linker)
elapsed = timeit.default_timer() - start_time
print('Seconds for single query: ' + str(elapsed))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


NER model has found 93 of the 101 entities.


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:202: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


RL preselection has found 84 of the 101 matches.


INFO:dedupe.blocking:10000, 0.2243362 seconds
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:223: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


RL model has matched 49 of the 101 matches.
RL model has made a total of 61 matches
Seconds for single query: 2049.9911239000003


In [ ]:
# prev time 1991.2779954999996

In [76]:
start_time = timeit.default_timer()
x = Pipeline(notary[notary.uuid.isin(final_df[final_df.vocop_match != '0'].uuid)], voc, nlp2, linker)
elapsed = timeit.default_timer() - start_time
print('Seconds for single query: ' + str(elapsed))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


NER model has found 93 of the 101 entities.


Empty DataFrame
Columns: [VOCOP_id, fullNameOriginal, firstNameOriginal, patronymicOriginal, familyNamePrefixOriginal, familyNameOriginal, placeOfOrigin, toponymCountryCode, countryCode, URI, LAT, LON, country/region, wageMin, rank, wageMax, wageStandard_DASminSupplementedBeini, HISCO_CODE, HISCO_URI, dateBeginService, dateEndServiceNoZeros, reasonEndService, endServiceWhere, shipOutward, shipID, shipTonnage, voyageID, voyNumberDAS, monthLetter, debtLetter, generalRemark, boardedAtCape, boardedAtCapeShipName, boardedAtCapeChamber, boardedAtCapeVoyageID, boardedAtCapeVoyNumberDAS, boardedAtCapeShipID, shipReturn, DASvoyageNumberReturnJourney, DASvoyageReturnID, DASshipReturnID, DASshipReturnTonnage, sourceReference, scanPermalink, cluster_ID, disambiguated, in_unique_set, in_clustered_set, ship_outward_year, date_end_service_DAS, date_end_service_improved, firstNameNormalized, familyNameNormalized, familyNamePrefixNormalized, patronymicNormalized, fullNameNormalized, placeOfOriginNormal

        VOCOP_id fullNameOriginal firstNameOriginal patronymicOriginal  \
271982    764045    Pieter Kleijn            Pieter                NaN   
277221    784359     Pieter Klijn            Pieter                NaN   
284049    802002     Pieter Klijn            Pieter                NaN   
288576    814032     Pieter Klijn            Pieter                NaN   

       familyNamePrefixOriginal familyNameOriginal placeOfOrigin  \
271982                      NaN             Kleijn      Holstijn   
277221                      NaN              Klijn    Dusseldorp   
284049                      NaN              Klijn     Amsterdam   
288576                      NaN              Klijn     Amsterdam   

       toponymCountryCode countryCode                                    URI  \
271982        Holstein DE          DE  https://www.wikidata.org/wiki/Q111224   
277221      Düsseldorf DE          DE       http://sws.geonames.org/2934246/   
284049       Amsterdam NL          NL       http

        VOCOP_id fullNameOriginal firstNameOriginal patronymicOriginal  \
277314    784452    Hendrik Staal           Hendrik                NaN   
468538   1188053     Hendrik Taal           Hendrik                NaN   

       familyNamePrefixOriginal familyNameOriginal   placeOfOrigin  \
277314                      NaN              Staal       Amsterdam   
468538                      NaN               Taal  Katwijk op zee   

        toponymCountryCode countryCode                               URI  ...  \
277314        Amsterdam NL          NL  http://sws.geonames.org/2759794/  ...   
468538  Katwijk aan Zee NL          NL  http://sws.geonames.org/2753010/  ...   

        year_end_service_DAS  year_end_service_improved  \
277314                   NaN                     1744.0   
468538                1746.0                     1746.0   

       date_begin_service_complete  date_end_service_complete  \
277314                  1744-01-01                 1744-11-02   
468538        

...

        VOCOP_id fullNameOriginal firstNameOriginal patronymicOriginal  \
461061   1180512         Jan Helt               Jan                NaN   

       familyNamePrefixOriginal familyNameOriginal placeOfOrigin  \
461061                      NaN               Helt     Amsterdam   

       toponymCountryCode countryCode                               URI  ...  \
461061       Amsterdam NL          NL  http://sws.geonames.org/2759794/  ...   

        year_end_service_DAS  year_end_service_improved  \
461061                   NaN                     1752.0   

       date_begin_service_complete  date_end_service_complete  \
461061                  1750-10-06                 1752-01-01   

       cape_boarder_is_linked  linked_vocop_id      cape_boarder_rule  \
461061                    NaN              NaN  no_cape_boarder_issue   

           dutch_rank converted_begin_date   converted_end_date  
461061  Onderstuurman  1750-10-06 00:00:00  1752-01-01 00:00:00  

[1 rows x 70 columns]
E

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:208: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


RL preselection has found 84 of the 101 matches.


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:229: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


RL model has matched 49 of the 101 matches.
RL model has made a total of 50 matches
Seconds for single query: 151.16825789999984


In [75]:
start_time = timeit.default_timer()
x = Pipeline(notary[notary.uuid.isin(final_df.loc[testje.cluster_id.to_list()][final_df.vocop_match!='0'].uuid)], voc, nlp2, linker)
elapsed = timeit.default_timer() - start_time
print('Seconds for single query: ' + str(elapsed))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  



NER model has found 29 of the 101 entities.


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


        VOCOP_id fullNameOriginal firstNameOriginal patronymicOriginal  \
287065    810136    Pieter Jurgen            Pieter                NaN   
330569   1049048    Pieter Jurgen            Pieter                NaN   
453005   1172393    Pieter Jurgen            Pieter                NaN   

       familyNamePrefixOriginal familyNameOriginal placeOfOrigin  \
287065                      NaN             Jurgen   Hadersleven   
330569                      NaN             Jurgen    Sanderburg   
453005                      NaN             Jurgen         Funen   

       toponymCountryCode countryCode                               URI  ...  \
287065       Haderslev DK          DK  http://sws.geonames.org/2620964/  ...   
330569      Sønderborg DK          DK  http://sws.geonames.org/2613102/  ...   
453005           Funen DK          DK  http://sws.geonames.org/2621748/  ...   

        year_end_service_DAS  year_end_service_improved  \
287065                   NaN                     1

        VOCOP_id     fullNameOriginal firstNameOriginal patronymicOriginal  \
190284    439690  Johan Hendrik Elaas             Johan            Hendrik   

       familyNamePrefixOriginal familyNameOriginal placeOfOrigin  \
190284                      NaN              Elaas  Wolfenbuttel   

       toponymCountryCode countryCode                               URI  ...  \
190284    Wolfenbüttel DE          DE  http://sws.geonames.org/2806914/  ...   

        year_end_service_DAS  year_end_service_improved  \
190284                   NaN                     1747.0   

       date_begin_service_complete  date_end_service_complete  \
190284                  1747-05-14                 1747-08-13   

       cape_boarder_is_linked  linked_vocop_id      cape_boarder_rule  \
190284                    NaN              NaN  no_cape_boarder_issue   

       dutch_rank converted_begin_date   converted_end_date  
190284    Soldaat  1747-05-14 00:00:00  1747-08-13 00:00:00  

[1 rows x 70 columns]
E

...

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:208: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


RL preselection has found 22 of the 101 matches.



RL model has matched 14 of the 101 matches.
RL model has made a total of 14 matches
Seconds for single query: 72.67109609999989


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:229: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


NameError: name 'a' is not defined

In [197]:
henk = x
good = []
c = 0
for x in henk:
    notid = henk[x][0].split('_', 1)[0]
    rows = final_df[(final_df.uuid == notid) & (final_df.vocop_match != '0')]
    for thing in rows.itertuples():
        if str(voc.loc[ast.literal_eval(thing.vocop_match)[1]].VOCOP_id) == str(henk[x][1]):
#             print(str(voc.loc[ast.literal_eval(thing.vocop_match)[1]].VOCOP_id))
#             print(str(henk[x][1]))
#             print('__________________________________')
            c += 1
            good.append(x)
for x in henk:
    if x not in good:
        notid = henk[x][0].split('_', 1)[0]
        print(voc[voc.VOCOP_id == int(henk[x][1])].fullNameOriginal.iloc[0])
        print(str(henk[x][1]))
        print('|||||||||||||||||||||||')
        rows = final_df[(final_df.uuid == notid)]
        for thing in rows.itertuples():
            if thing.vocop_match != '0':
                print(str(voc.loc[ast.literal_eval(thing.vocop_match)[1]].VOCOP_id), thing.name)
            else:
                print(str(thing.vocop_match), thing.name)

        print('________________________________________')
print(c)

Christiaan Andries
1089739
|||||||||||||||||||||||
1045147 Claas Pietersz
0 Christiaan Andriesz
0 Anthonij van den Hoed
0 Thomas Schippers
________________________________________
Abraham van der Mijn
1177397
|||||||||||||||||||||||
1067268 Leendert Zuijderland
0 Sophia Magdalena Slusseren
0 Florus van Reessen
0 Abraham van der Naa
0 Eldert Otten
________________________________________
Willem Kemp
1178800
|||||||||||||||||||||||
0 Elias Schippers
0 Willem de Kemp
326331 Hendrik Staanke
________________________________________
Hendrik Kloek
1077430
|||||||||||||||||||||||
0 Jacobus Braambergen
1092795 Hermanus Soet
0 Hendrik Soet
0 Claas Soet
________________________________________
Hendrik Hardelant
1133405
|||||||||||||||||||||||
0 Jan Davidsz
0 Marten Raat
0 Hendrik Haan
330316 Jan Smit
________________________________________
Johan Hilmers
834597
|||||||||||||||||||||||
0 Johan Dolmer
837974 Hendrik Burger
________________________________________
54


In [150]:
c

56

In [177]:
final_df

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
index,,,,,,,,,,,,,,,,
0,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,NaN,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Trijntje Lansink,0,0,0
1,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,NaN,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Martinus Ciprianus,0,0,0
2,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,NaN,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Simon van Zuijderwijk,0,0,0
3,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,NaN,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Margareta van Zuijdewijk,0,0,0
4,1b037028-bea5-0fa8-e6c5-68155f5f2b21,358,JAN VERLEIJ,11885,108046,Attestatie,1741-07-26,nederlands,\nordentelijke vrijage\n,"[{'voornaam': 'Gerard', 'tussenvoegsel': None,...","['KLAB05372000193.JPG', 'KLAB05372000194.JPG',...",Verklaing gepasseert den 26 Julij 1741No: 60- ...,Gerard Plaatman,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3129,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,NaN,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Coenraad Smit,"[{'index': 83363, 'name_original': 'Coenraad S...","{'83363': {'ships': ['nan'], 'rank': [], 'loca...",0
3130,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,NaN,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Johann Wilhelm Schmittener,0,0,0
3131,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,NaN,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Jochem Matthijs Smit,0,0,0


In [78]:
notary[notary.uuid == '40907d8c-4dfc-54a8-de2b-2ecb6b24a82c'].iloc[0].text

"No: 420 Procuratie gepasseert den 2: Septembr: 1746SijtOp heden den 2 feplemta des jaars 1746 Comparede voor mij Jan herleid Notaris publicq bij den Ed hove van Holland geadmitteert binnen Amsterdam Residerende d' E Mebehertden Septeb: 1555Compareerde d' E. Melchert Folmer van Dragten In den Jaaren 1759 het Schip abbekerk Voo d Came Zeeland uijtgevaaren en de nu ontangs met het Schip hujsenduijn voor de Camr Rotterdam uijtoost, ndien te rig gekoomen en behoorlijk aff gedankz de welke verklaarde sen Te Contitueeren d' E E Willem de kempbode van Amst op zegland en Tomas Schippers wonende tot middelbing te Samen & ider in 't bisonder peehalijk Omme te verderen en ont kangen van de Ed Heeren bewind hebbenen van de boostijndesehe Compagnie de r Camer Zeeland roornt alle 't Geenenhij Comrt voor gagier en maand geld off anderzints Mitsgaders hetdoueer ofl Eouwalent als mede voor premie wegens agtekomkomen volgens het Reglement van At W4a ter Camere voornt te goeden st heeft voor den ontfang 

In [80]:
voc[voc.VOCOP_id == 253559].shipReturn

62211    HUIS TER DUINE
Name: shipReturn, dtype: object

In [71]:
start_time = timeit.default_timer()
x = Pipeline(notary[notary.uuid.isin(final_df.uuid)][0:100], voc, nlp2, linker)
elapsed = timeit.default_timer() - start_time
print('Seconds for single query: ' + str(elapsed))


NER model has found 6 of the 101 entities.


KeyboardInterrupt: 

In [ ]:
start_time = timeit.default_timer()
x = Pipeline(notary[notary.uuid.isin(final_df.uuid)][1300:1950], voc, nlp2, 'rl models/50no')
elapsed = timeit.default_timer() - start_time
print('Seconds for single query: ' + str(elapsed))

In [40]:
testje = pd.read_csv('rl models/test_notary_old.csv')

In [41]:
final_df = pd.read_csv('final_df.csv', index_col=['index'])

In [35]:
final_df.loc[testje.cluster_id.to_list()][final_df.vocop_match!='0']

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
index,,,,,,,,,,,,,,,,
5857,a517338b-8389-d6f7-18c8-29b1f8f944fc,358,JAN VERLEIJ,11957,29685,Machtiging,1765-07-06,nederlands,\nJoseph vertrok in 1761 aan boord van het VOC...,"[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB06471000251.JPG', 'KLAB06471000252.JPG',...",No: 195 Procuratie gecasseerd den 6e: Julij 17...,Jan Joseph,"[{'index': 5295, 'name_original': 'Jan Josephs...","{'5295': {'ships': [], 'rank': [], 'location':...","['Jan Joseph', 81104]"
5261,ee812a50-7186-0414-83ff-f2d6682b674e,358,JAN VERLEIJ,11948,33801,Machtiging,1762-06-16,nederlands,\nOhlandt vertrok in 1761 met het VOC-schip Bo...,"[{'voornaam': 'Teunis', 'tussenvoegsel': None,...","['KLAB06151000089.JPG', 'KLAB06151000090.JPG',...",No: 174 Procuratie gepasseert den 16e: Junij 1...,Teunis Ohlandt,"[{'index': 80326, 'name_original': 'Teunis Ohl...","{'80326': {'ships': ['Borssele'], 'rank': [], ...","['Teunis Ohlant', 80326]"
2282,b30c8a95-e5a2-f818-da9f-d19ec93d59e6,358,JAN VERLEIJ,11910,17760,Machtiging,1750-04-17,nederlands,"\nSchip de Ouderkerk, schip de Standvastigheid\n","[{'voornaam': 'Johannes', 'tussenvoegsel': 'va...","['KLAB05289000441.JPG', 'KLAB05289000442.JPG',...",No: 137 Procuratie gepasseert den 17e: april 1...,Jan Helt,"[{'index': 415, 'name_original': 'Jan Held', '...","{'415': {'ships': [], 'rank': [], 'location': ...","['Jan Helt', 417031]"
7038,1bb353b6-2942-475c-f9f0-8e187afb5c9c,358,JAN VERLEIJ,11891,11202,Obligatie,1744-12-11,nederlands,\nschip Schellag\n,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05426000424.JPG', 'KLAB05426000425.JPG',...",No 831 Clegatie Gepasseerde Den 10 docem ber 1...,Johan Christiaan Rigter,"[{'index': 39258, 'name_original': 'Johan Chri...","{'39258': {'ships': [], 'rank': [], 'location'...","['Johan Christiaan Richter', 379155]"
54466,da6f9044-1588-3bf2-1e2f-72b7b374e3db,358,JAN VERLEIJ,11905,18207,Machtiging,1748-09-19,nederlands,"\nschip 't Huys te Spijk, VOC kamer Zeeland; s...","[{'voornaam': 'Cornelis', 'tussenvoegsel': Non...","['KLAB05559000102.JPG', 'KLAB05559000103.JPG']",No: 259 Procuratie gepoassseert den 19e: Septe...,Jan Govertsz,"[{'index': 21034, 'name_original': 'Jan Govers...","{'21034': {'ships': [], 'rank': [], 'location'...","['Jan Govertsz.', 66979]"
54465,da6f9044-1588-3bf2-1e2f-72b7b374e3db,358,JAN VERLEIJ,11905,18207,Machtiging,1748-09-19,nederlands,"\nschip 't Huys te Spijk, VOC kamer Zeeland; s...","[{'voornaam': 'Cornelis', 'tussenvoegsel': Non...","['KLAB05559000102.JPG', 'KLAB05559000103.JPG']",No: 259 Procuratie gepoassseert den 19e: Septe...,Cornelis Vis,"[{'index': 33017, 'name_original': 'Cornelis V...","{'33017': {'ships': [], 'rank': [], 'location'...","['Cornelis Vis', 66931]"
5964,1bf4e148-14a4-dd4d-ef24-503a157766dc,358,JAN VERLEIJ,11960,21944,Machtiging,1766-08-25,nederlands,"\nVOC schip Damzigt, schip Walcheren, innen va...","[{'voornaam': 'Pieter', 'tussenvoegsel': 'van'...","['KLAB06468000354.JPG', 'KLAB06468000355.JPG',...",an No: 215 Procuratie gecasseerd den 25e: aug:...,Hendrik Dames,"[{'index': 55709, 'name_original': 'Hendrik Da...","{'55709': {'ships': [], 'rank': [], 'location'...","['Hendrik Dames', 80776]"
5846,934d3c19-5d20-33d8-5826-9f46658b64b2,358,JAN VERLEIJ,11957,28686,Machtiging,1765-07-01,nederlands,\nSchip Overschie; schip de Gouverneur Generaal\n,"[{'voornaam': 'Ernst August', 'tussenvoegsel':...","['KLAB06471000222.JPG', 'KLAB06471000223.JPG']",en odt Conden de hem dezeegeld den 2Jeaf 1755 ...,Ernst August Berkelman,"[{'index': 308820, 'name_original': 'Ernst Aug...","{'308820': {'ships': [], 'rank': [], 'location...","['Ernst Augustus Berkelman', 447428]"
5044,14329821-8d12-2988-6ab8-b243be1bd6fc,358,JAN VERLEIJ,11946,21481,Machtiging,1761-11-14,nederlands,\nVOC schip admiraal de Ruijter\n,"[{'voornaam': 'Johannes', 'tussenvoegsel': Non...","['KLAB06180000163.JPG', 'KLAB06180000164.JP

[7280,
 6320,
 3210,
 9918,
 2068,
 690,
 54929,
 3127,
 5550,
 2157,
 54814,
 3828,
 3082,
 3111,
 5308,
 5675,
 897,
 5578,
 3737,
 54122,
 6594,
 6353,
 6784,
 6158,
 5366,
 54714,
 2039,
 3787,
 54187,
 6202,
 4214,
 734,
 54457,
 2447,
 9671,
 7157,
 5857,
 698,
 9094,
 5017,
 54344,
 3564,
 9390,
 54190,
 9167,
 2385,
 3535,
 2403,
 8727,
 5199,
 5423,
 9995,
 3315,
 3518,
 4056,
 9581,
 4005,
 9580,
 54362,
 1645,
 3490,
 6451,
 2387,
 2584,
 601,
 54215,
 3658,
 6807,
 5486,
 7224,
 8880,
 54245,
 201,
 212,
 9290,
 1632,
 3931,
 7336,
 7018,
 1283,
 3361,
 54405,
 4138,
 7557,
 7005,
 2040,
 3468,
 3213,
 3129,
 3603,
 5052,
 54103,
 54186,
 8523,
 5164,
 8506,
 9218,
 7517,
 5701,
 6373,
 3888,
 6111,
 9772,
 3372,
 1811,
 8984,
 8864,
 8571,
 8373,
 2407,
 7536,
 3586,
 2976,
 7486,
 1911,
 605,
 1257,
 6597,
 8751,
 2120,
 2817,
 9115,
 2032,
 54233,
 7345,
 8417,
 7360,
 1272,
 3402,
 2135,
 8618,
 7339,
 390,
 786,
 567,
 6318,
 7092,
 958,
 4127,
 9055,
 3842,
 54646,
 1

In [ ]:
def clustering(model_dir, notary, voc):
    start_time = timeit.default_timer()

    settings_file = model_dir

    def preProcess(column):
        """
        Do a little bit of data cleaning with the help of Unidecode and Regex.
        Things like casing, extra spaces, quotes and new lines can be ignored.
        """

        column = unidecode(column)
        column = re.sub('\n', ' ', column)
#         column = re.sub('-', '', column)
        column = re.sub('/', ' ', column)
        column = re.sub("'", '', column)
        column = re.sub(",", '', column)
        column = re.sub(":", ' ', column)
        column = re.sub('  +', ' ', column)
        column = column.strip().strip('"').strip("'").lower().strip()
        if not column:
            column = None
        return column


    def readData(filename):
        """
        Read in our data from a CSV file and create a dictionary of records,
        where the key is a unique record ID.
        """

        data_d = {}

        with open(filename) as f:
            reader = csv.DictReader(f)
            for i, row in enumerate(reader):
                clean_row = dict([(k, preProcess(v)) for (k, v) in row.items()])
                data_d[clean_row['index']] = dict(clean_row)
        return data_d
    
    print('importing data ...')
    data_1 = readData(notary)
    data_2 = readData(voc)
    
    elapsed = timeit.default_timer() - start_time
    print('Reading data: ' + str(elapsed))
    start_time = timeit.default_timer()

    def descriptions():
        for dataset in (data_1, data_2):
            for record in dataset.values():
                yield record['description']

    # ## Training

    if os.path.exists(settings_file):
        print('reading from', settings_file)
        with open(settings_file, 'rb') as sf:
            linker = dedupe.StaticRecordLink(sf)

    else:
        raise ValueError('Could not find model')
    
    elapsed = timeit.default_timer() - start_time
    print('Loading Model: ' + str(elapsed))
    start_time = timeit.default_timer()

    # ## Blocking

    # ## Clustering

    # Find the threshold that will maximize a weighted average of our
    # precision and recall.  When we set the recall weight to 2, we are
    # saying we care twice as much about recall as we do precision.
    #
    # If we had more data, we would not pass in all the blocked data into
    # this function but a representative sample.

    print('clustering...')
    try:
        linked_records = linker.join(data_1, data_2, 0.3)
    except:
        print('No matches made')
        return None
    
    print('# duplicate sets', len(linked_records))
    # ## Writing Results
    elapsed = timeit.default_timer() - start_time
    print('Clustering: ' + str(elapsed))
    # Write our original data back out to a CSV with a new column called
    # 'Cluster ID' which indicates which records refer to each other.

    cluster_membership = {}
    for cluster_id, (cluster, score) in enumerate(linked_records):
        for record_id in cluster:
            cluster_membership[record_id] = {'Cluster ID': cluster_id,
                                             'Link Score': score}
    clusters = {}
    for x in cluster_membership:
        if cluster_membership[x]['Cluster ID'] not in clusters:
            clusters[cluster_membership[x]['Cluster ID']] = x
        else:
            clusters[cluster_membership[x]['Cluster ID']] = [clusters[cluster_membership[x]['Cluster ID']], x]
    
    matches = clusters
    return matches


In [ ]:
clustering('rl models/50no', 'converted_notary.csv', 'converted_voc.csv')

### NEL

In [ ]:
def NEL(entity, knowledgebase, model_dir):
    ''' Takes in a string containing an entity and returns either a match within the specified pandas 
        knowledgebase or None if no match is present.
    ''' 
    # Find and narrow down matches
    possible_matches = find_matches(entity.naam, knowledgebase, 'fullNameOriginal')
    name_dict = knowledgebase.fullNameNormalized.value_counts()
    converted_matches = convert_matches(entity, possible_matches, name_dict)
    if converted_matches == {}:
        return None
    ranked_matches = ranking(converted_matches, model_dir)
    return max(ranked_matches)

def convert_matches(entity, matches, name_dict):
    
    holder = {}
    notary_date = datetime.strptime(entity.datering, '%Y-%m-%d')
    
    # Try to convert the dates for each entry into datetime format
    for x in range(matches.shape[0]):
        try:
            date1 = datetime.strptime(matches['date_begin_service_complete'].iloc[x], '%Y-%m-%d')
        except:
            date1 = datetime(year=1, month=1, day =1 )
        try:
            date2 = datetime.strptime(matches['date_end_service_complete'].iloc[x], '%Y-%m-%d')
        except:
            date2 = datetime(year=1, month=1, day =1 )
            
        # Keep only matches that are within distance days from notary_date
        if abs((notary_date - date1).days) < 90 or abs((notary_date - date2).days) < 90:
            name_ratio = fuzz.ratio(entity.naam, matches['fullNameOriginal'].iloc[x])
            name_count = name_dict[matches['fullNameNormalized'].iloc[x]]
            vocid=matches['VOCOP_id'].iloc[x]
            day_dif = min([abs((notary_date - date1).days), abs((notary_date - date2).days)])
            location = len(match_finder(entity.text, [matches['placeOfOrigin'].iloc[x]], 80))
            rank = len(match_finder(entity.text, [matches['dutch_rank'].iloc[x]], 80))
            numships = len(match_finder(entity.text, [matches['shipOutward'].iloc[x], matches['shipReturn'].iloc[x]], 80))
            holder[vocid] = [name_ratio, name_count, day_dif, location, rank, numships]

    return holder

def serialize_example(values):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    def _float_feature(value):
        """Returns an float_list from a int/float."""
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    
    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.
    feature = {}
    for x in enumerate(values):
        feature[str(x[0] + 1)] = _float_feature(values[x[0]])

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def ranking(matches, directory):
    tags=["serve"]
    signature_def_key = "predict"
    saved_model_dir = directory
    holder = []
    with Session() as sess:
        loader.load(sess, tags, saved_model_dir)
        serialized_examples = []
        vocop_ids = [x for x in matches]
        for x in matches:
            names = matches[x][0]
            days = matches[x][2]
            locations = matches[x][3]
            ranks = matches[x][4]
            ships = matches[x][5]
            name_count = matches[x][1]
            serialized_example = serialize_example([names, name_count, days, locations, 
                                                   ranks, ships] )
            serialized_examples.append(serialized_example)
            inputs_feed_dict = {'input_example_tensor:0': serialized_examples}
            outputs = sess.run('groupwise_dnn_v2/accumulate_scores/div_no_nan:0', feed_dict=inputs_feed_dict)
            output = [(outputs[x], vocop_ids[x]) for x in range(len(outputs))]
            holder = output
    return holder